In [1]:
# =============================================================================
# Import OHLCV data and transform it to Renko
# Author : Mayank Rasu (http://rasuquant.com/wp/)

# Please report bug/issues in the Q&A section
# =============================================================================

# Import necesary libraries
import yfinance as yf
from stocktrends import Renko

# Download historical data for required stocks
tickers = ["AMZN","GOOG","MSFT"]
ohlcv_data = {}
hour_data = {}
renko_data = {}

# looping over tickers and storing OHLCV dataframe in dictionary
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp
    temp = yf.download(ticker,period='1y',interval='1h')
    temp.dropna(how="any",inplace=True)
    hour_data[ticker] = temp
    
def ATR(DF, n=14):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean()
    return df["ATR"]

def renko_DF(DF, hourly_df):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df.drop("Close",axis=1,inplace=True)
    df.columns = ["date","open","high","low","close","volume"]
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0)
    renko_df = df2.get_ohlc_data() #if using older version of the library please use get_bricks() instead
    return renko_df

for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker])

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
ohlcv_data

{'AMZN':                                  Open        High         Low       Close  \
 Datetime                                                                    
 2024-04-23 10:55:00-04:00  177.859695  178.199997  177.824997  178.100006   
 2024-04-23 11:00:00-04:00  178.110001  178.149994  177.320007  177.444702   
 2024-04-23 11:05:00-04:00  177.449997  177.630005  177.410004  177.559998   
 2024-04-23 11:10:00-04:00  177.589996  178.139999  177.589996  178.110107   
 2024-04-23 11:15:00-04:00  178.130005  178.330002  178.029999  178.139999   
 ...                               ...         ...         ...         ...   
 2024-05-23 10:35:00-04:00  182.740097  183.233002  182.740097  183.220001   
 2024-05-23 10:40:00-04:00  183.219894  183.304993  182.910004  182.945007   
 2024-05-23 10:45:00-04:00  182.964996  183.349197  182.940094  183.300003   
 2024-05-23 10:50:00-04:00  183.300003  183.330002  182.759995  182.761307   
 2024-05-23 10:55:00-04:00  182.759995  182.832993  182.

In [8]:
renko_data['AMZN']

,date,open,high,low,close,uptrend
0,2024-04-23 10:55:00-04:00,174.0,177.0,174.0,177.0,True
1,2024-04-24 09:35:00-04:00,177.0,180.0,177.0,180.0,True
2,2024-04-25 09:30:00-04:00,177.0,177.0,174.0,174.0,False
3,2024-04-25 09:30:00-04:00,174.0,174.0,171.0,171.0,False
4,2024-04-26 09:35:00-04:00,174.0,177.0,174.0,177.0,True
5,2024-04-26 12:35:00-04:00,177.0,180.0,177.0,180.0,True
6,2024-05-01 14:40:00-04:00,180.0,183.0,180.0,183.0,True
7,2024-05-03 09:30:00-04:00,183.0,186.0,183.0,186.0,True
8,2024-05-07 09:40:00-04:00,186.0,189.0,186.0,189.0,True
9,2024-05-21 09:30:00-04:00,186.0,186.0,183.0,183.0,False


In [5]:
import pandas as pd

{'AMZN':                         date   open   high    low  close  uptrend
 0  2024-04-23 10:55:00-04:00  174.0  177.0  174.0  177.0     True
 1  2024-04-24 09:35:00-04:00  177.0  180.0  177.0  180.0     True
 2  2024-04-25 09:30:00-04:00  177.0  177.0  174.0  174.0    False
 3  2024-04-25 09:30:00-04:00  174.0  174.0  171.0  171.0    False
 4  2024-04-26 09:35:00-04:00  174.0  177.0  174.0  177.0     True
 5  2024-04-26 12:35:00-04:00  177.0  180.0  177.0  180.0     True
 6  2024-05-01 14:40:00-04:00  180.0  183.0  180.0  183.0     True
 7  2024-05-03 09:30:00-04:00  183.0  186.0  183.0  186.0     True
 8  2024-05-07 09:40:00-04:00  186.0  189.0  186.0  189.0     True
 9  2024-05-21 09:30:00-04:00  186.0  186.0  183.0  183.0    False,
 'GOOG':                          date   open   high    low  close  uptrend
 0   2024-04-23 10:55:00-04:00  156.0  159.0  156.0  159.0     True
 1   2024-04-26 09:30:00-04:00  159.0  162.0  159.0  162.0     True
 2   2024-04-26 09:30:00-04:00  162.0  165